In [20]:
!mkdir -p ./kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [22]:
!kaggle datasets download -d soumikrakshit/lol-dataset

Dataset URL: https://www.kaggle.com/datasets/soumikrakshit/lol-dataset
License(s): unknown
 98% 326M/331M [00:02<00:00, 167MB/s]
100% 331M/331M [00:02<00:00, 169MB/s]


In [23]:
!unzip /content/lol-dataset.zip -d /content/

Archive:  /content/lol-dataset.zip
  inflating: /content/lol_dataset/eval15/high/1.png  
  inflating: /content/lol_dataset/eval15/high/111.png  
  inflating: /content/lol_dataset/eval15/high/146.png  
  inflating: /content/lol_dataset/eval15/high/179.png  
  inflating: /content/lol_dataset/eval15/high/22.png  
  inflating: /content/lol_dataset/eval15/high/23.png  
  inflating: /content/lol_dataset/eval15/high/493.png  
  inflating: /content/lol_dataset/eval15/high/547.png  
  inflating: /content/lol_dataset/eval15/high/55.png  
  inflating: /content/lol_dataset/eval15/high/665.png  
  inflating: /content/lol_dataset/eval15/high/669.png  
  inflating: /content/lol_dataset/eval15/high/748.png  
  inflating: /content/lol_dataset/eval15/high/778.png  
  inflating: /content/lol_dataset/eval15/high/780.png  
  inflating: /content/lol_dataset/eval15/high/79.png  
  inflating: /content/lol_dataset/eval15/low/1.png  
  inflating: /content/lol_dataset/eval15/low/111.png  
  inflating: /content/l

In [8]:
import os
import joblib
import numpy as np
from keras.preprocessing.image import img_to_array, load_img
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import argparse

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model,load_model,model_from_json
from tensorflow.keras.layers import Conv2D, Conv2DTranspose,GlobalAveragePooling2D, AveragePooling2D, MaxPool2D, UpSampling2D,BatchNormalization, Activation, ReLU, Flatten, Dense, Input
from tensorflow.keras.layers import Add, Multiply, Concatenate, Softmax,Reshape,Dropout
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import softmax
from tensorflow.keras.losses import MeanSquaredError
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.applications import VGG19
from tensorflow.image import ssim
tf.keras.backend.set_image_data_format('channels_last')
import keras.backend as K

**Serializing the Custom Library So that Json will deserialize them**

In [10]:
@tf.keras.saving.register_keras_serializable()
class Conv_block(tf.keras.layers.Layer):
    def  __init__(self, num_filters=200, kernel_size=3,**kwargs):
        super().__init__(**kwargs)
        self.num_filters=num_filters
        self.kernel_size=kernel_size
        self.conv_1 = Conv2D(filters=self.num_filters, kernel_size=self.kernel_size, padding='same')
        self.conv_2 = Conv2D(filters=self.num_filters, kernel_size=self.kernel_size, padding='same')
        self.conv_3 = Conv2D(filters=self.num_filters, kernel_size=self.kernel_size, padding='same')
        self.conv_4 = Conv2D(filters=self.num_filters, kernel_size=self.kernel_size, padding='same')
        self.SE     = SEBlock(filters=self.num_filters)
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'num_filters': self.num_filters,
            'kernel_size':self.kernel_size
        })
        return config

    def call(self, X):
        X = self.conv_1(X)
        X = ReLU()(X)

        X = self.conv_2(X)
        X = ReLU()(X)

        X = self.conv_3(X)
        X = ReLU()(X)

        X = self.SE(X)
        return X

In [11]:
@tf.keras.saving.register_keras_serializable()
class SEBlock(tf.keras.layers.Layer):
    def __init__(self, filters, reduction=16, **kwargs):
        super(SEBlock, self).__init__(**kwargs)
        self.filters = filters
        self.reduction = reduction
        self.global_avg_pool = GlobalAveragePooling2D()
        self.reshape = Reshape((1, 1, filters))
        self.dense1 = Dense(filters // reduction, activation='relu', kernel_initializer='he_normal', use_bias=False)
        self.dense2 = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)

    def call(self, input_tensor):
        se = self.global_avg_pool(input_tensor)
        se = self.reshape(se)
        se = self.dense1(se)
        se = self.dense2(se)
        return Multiply()([input_tensor, se])

    def get_config(self):
        config = super(SEBlock, self).get_config()
        config.update({
            'filters': self.filters,
            'reduction': self.reduction
        })
        return config

In [12]:
@tf.keras.saving.register_keras_serializable()
class DWT_downsampling(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, x):
        x1 = x[:, 0::2, 0::2, :] #x(2i−1, 2j−1)
        x2 = x[:, 1::2, 0::2, :] #x(2i, 2j-1)
        x3 = x[:, 0::2, 1::2, :] #x(2i−1, 2j)
        x4 = x[:, 1::2, 1::2, :] #x(2i, 2j)

        x_LL = x1 + x2 + x3 + x4
        x_LH = -x1 - x3 + x2 + x4
        x_HL = -x1 + x3 - x2 + x4
        x_HH = x1 - x3 - x2 + x4

        return Concatenate(axis=-1)([x_LL, x_LH, x_HL, x_HH])

In [13]:
@tf.keras.saving.register_keras_serializable()
class IWT_upsampling(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, x):
        x_LL = x[:, :, :, 0:x.shape[3]//4]
        x_LH = x[:, :, :, x.shape[3]//4:x.shape[3]//4*2]
        x_HL = x[:, :, :, x.shape[3]//4*2:x.shape[3]//4*3]
        x_HH = x[:, :, :, x.shape[3]//4*3:]

        x1 = (x_LL - x_LH - x_HL + x_HH)/4
        x2 = (x_LL - x_LH + x_HL - x_HH)/4
        x3 = (x_LL + x_LH - x_HL - x_HH)/4
        x4 = (x_LL + x_LH + x_HL + x_HH)/4

        y1 = K.stack([x1,x3], axis=2)
        y2 = K.stack([x2,x4], axis=2)
        shape = K.shape(x)
        return K.reshape(K.concatenate([y1,y2], axis=-1), K.stack([shape[0], shape[1]*2, shape[2]*2, shape[3]//4]))

In [14]:
vgg = VGG19(include_top=False, weights='imagenet', input_shape=(256,256,3))
vgg.trainable = False
def perceptual_loss(y_true, y_pred):
    true_features=vgg(y_true)
    pred_features=vgg(y_pred)
    return MeanSquaredError()(true_features, pred_features)

80134624/80134624 [==============================] - 1s 0us/step


In [52]:
def psnr(y_true, y_pred):
    max_pixel = 1.0
    return tf.image.psnr(y_true, y_pred, max_val=max_pixel)

In [16]:
def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(ssim(y_true, y_pred, max_val=1.0))

In [17]:
def net_loss(y_pred,y_true):
  perceptual=perceptual_loss(y_true=y_true,y_pred=y_pred)
  ssim=ssim_loss(y_true=y_true,y_pred=y_pred)
  return ssim + perceptual

In [35]:
def load_images_from_folder(folder):
    images_list=[]
    filenames=[]
    for filename in os.listdir(folder):
      img = load_img(os.path.join(folder, filename),target_size=(256,256))
      if img is not None:
          images_list.append(np.array(img)/255.0)
          filenames.append(filename)
    return images_list, filenames

In [40]:
def save_images_to_folder(folder, images, filenames):
    os.makedirs(folder, exist_ok=True)
    for img, filename in zip(images, filenames):
        img = Image.fromarray((img*255).astype('uint8'))
        img = img.resize((600,400),Image.BILINEAR)
        img.save(os.path.join(folder, filename))

**Call this function and pass the path of the base directory and model path.**

In [37]:
base_path='/content/lol_dataset/'

In [38]:
def test_model(base_path,model_path):
  test_path=os.path.join(base_path,'eval15/low')
  predicted_path=os.path.join(base_path,'predicted')
  # Load the trained model
  model_file=joblib.load(model_path)
  model=tf.keras.models.model_from_json(model_file['model_json'])
  model.set_weights(model_file['model_weights'])

  #Loading the test Images
  test_images, filenames = load_images_from_folder(test_path)

  # Predicting the test images with the model
  predicted_images = [model.predict(np.expand_dims(img, axis=0))[0] for img in test_images]

  #Saving the Predicted Images
  save_images_to_folder(predicted_path, predicted_images, filenames)

  print(f"Saved the Predicted Image to path {predicted_path}")

In [ ]:
test_model(base_path,'/content/drive/MyDrive/FINAL_MODEL.joblib')